# SEND EVENTS TO CONTAINER

In [ ]:
event_hub_namespace="iesstsabbadbaa-grp-06-10"
consumer_eventhub_connection_str='Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=ridesconsumer;SharedAccessKey=f82trXEoV+VyuCL2Xah3fSOB9JWoaG36L+AEhJ9CqaE=;EntityPath=ride-requests-g8-v2'
eventhub_name='ride-requests-g8-v2'

consumer_eventhub_connection_str_requests='Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=driversconsumer;SharedAccessKey=+xc2QnEHgB6Cr2L47Usi+OC+m3xwxLc7H+AEhNZAStI=;EntityPath=drivers-g8-v2'
eventhub_name_requests='drivers-g8-v2'

# Azure Blob
account_name='iesstsabbadbaa'
account_key='YiUournbsPdwTGVanY1bNSsUcy1Z7NUNoty1fo0SDxcHoe6gzTxeq7BlIG4c3owhT1IIhim2IsHF+AStMie8jw=='
container_name='streamed-data-group8'

In [ ]:
import os
import subprocess

# Fetch the latest Spark 3.x.x version
# curl -s https://downloads.apache.org/spark/ → Fetches the Spark download page.
# grep -o 'spark-3\.[0-9]\+\.[0-9]\+' → Extracts only versions that start with spark-3. (ignoring Spark 4.x if it exists in the future).
# sort -V → Sorts the versions numerically.
# tail -1 → Selects the latest version.
spark_version = subprocess.run(
    "curl -s https://downloads.apache.org/spark/ | grep -o 'spark-3\\.[0-9]\\+\\.[0-9]\\+' | sort -V | tail -1",
    shell=True, capture_output=True, text=True
).stdout.strip()

spark_version

'spark-3.5.5'

In [ ]:
spark_release=spark_version
hadoop_version='hadoop3'

import os, time
start=time.time()
os.environ['SPARK_RELEASE']=spark_release
os.environ['HADOOP_VERSION']=hadoop_version
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_release}-bin-{hadoop_version}"

In [ ]:
if True:
  # Run below commands in google colab
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null # install Java8
  !wget -q http://apache.osuosl.org/spark/${SPARK_RELEASE}/${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # download spark-3.3.X
  !tar xf ${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # unzip it

  !pip install -q findspark # install findspark
  # findspark find your Spark Distribution and sets necessary environment variables

In [ ]:
import findspark
findspark.init()

# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.5.5


In [ ]:
ride_request_schema = """
{
  "type": "record",
  "name": "RideRequest",
  "fields": [
    {"name": "request_id",               "type": "string"},
    {"name": "passenger_id",             "type": "string"},
    {"name": "passenger_name",           "type": "string"},
    {"name": "driver_name",              "type": ["null", "string"]},
    {"name": "pickup_latitude",          "type": "double"},
    {"name": "pickup_longitude",         "type": "double"},
    {"name": "destination_latitude",     "type": "double"},
    {"name": "destination_longitude",    "type": "double"},
    {"name": "timestamp",                "type": "string"},
    {"name": "vehicle_type",             "type": "string"},
    {"name": "surge_multiplier",         "type": "float"},
    {"name": "traffic_delay_multiplier", "type": "float"},
    {"name": "pickup_distance_km",       "type": "float"},
    {"name": "pickup_eta_min",           "type": "float"},
    {"name": "distance_km",              "type": "float"},
    {"name": "estimated_ride_time_min",  "type": "float"},
    {"name": "fare_euros",               "type": "float"},
    {"name": "payment_method",           "type": "string"},
    {"name": "driver_rating_given",      "type": "float"}
  ]
}
"""

driver_profile_schema = """
{
  "type": "record",
  "name": "DriverProfile",
  "fields": [
    {"name": "driver_id",   "type": "string"},
    {"name": "driver_name", "type": "string"},
    {"name": "vehicle_type","type": "string"},
    {"name": "latitude",    "type": "double"},
    {"name": "longitude",   "type": "double"},
    {"name": "status",      "type": "string"},
    {"name": "shift_start", "type": "int"},
    {"name": "shift_duration","type": "int"}
  ]
}
"""


In [ ]:
# JARs needed for Hadoop-compatible access in Azure Storage
jar_dependencies= ",".join([
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0",
    "org.apache.spark:spark-avro_2.12:3.5.0",
    "org.apache.hadoop:hadoop-azure:3.3.1",          # Hadoop Azure connector
    "com.microsoft.azure:azure-storage:8.6.6"        # Azure Blob SDK dependency
])


In [ ]:
from pyspark.sql import SparkSession

account_name = "iesstsabbadbaa"
account_key  = "YiUournbsPdwTGVanY1bNSsUcy1Z7NUNoty1fo0SDxcHoe6gzTxeq7BlIG4c3owhT1IIhim2IsHF+AStMie8jw=="

spark = SparkSession \
    .builder \
    .appName("RideHailingDualStream") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,"
            "org.apache.spark:spark-avro_2.12:3.5.0,"
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config(f"fs.azure.account.key.{account_name}.dfs.core.windows.net", account_key) \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()


In [ ]:
#from pyspark.sql import SparkSession
#from pyspark.sql.avro.functions import from_avro

# Create a Spark session


#spark = SparkSession \
#    .builder \
#    .appName("StreamingAVROFromKafka") \
#    .config("spark.streaming.stopGracefullyOnShutdown", True) \
#    .config("spark.jars.packages", jar_dependencies) \
#    .config(f"fs.azure.account.key.{account_name}.blob.core.windows.net", account_key) \
#    .config("spark.sql.shuffle.partitions", 4) \
#    .master("local[*]") \
#    .getOrCreate()



In [ ]:
# Kafka Configuration for reading from Kafka/Event Hub
# Kafka source will create a unique group id for each query automatically. The user can set the prefix of the automatically
# generated group.id’s via the optional source option groupIdPrefix, default value is “spark-kafka-source”.
# Consumer groups in Kafka/Event Hubs are typically defined explicitly by clients (Kafka consumers),
# but Spark Structured Streaming manages consumer offsets internally, without explicitly registering or creating a visible consumer group within Azure Portal.
# groupIdPrefix auto-generates consumer group IDs for Kafka internally, but these DO NOT appear explicitly as consumer groups within the Azure Portal under
# Event Hub Consumer Groups.

kafkaConfRides_Req = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
    # Below settins required if kafka is secured, for example when connecting to Azure Event Hubs:
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{consumer_eventhub_connection_str}";',

    "subscribe": eventhub_name, # subscribe to the entire topic
    "startingOffsets": "latest", # "latest", "earliest", '{"topic_name: {"0": 62821}}'
    # "startingOffsets": f'{{"{eventhub_name}": {{"0": 212350, "1": -1, "2": 212152, "3": -1}}}}', # "latest", "earliest", '{"topic_name: {"0": 212350, "1": -1, "2": 212152, "3": -1}}' -1: latest, -2: earliest

    # "assign": f'{{"{eventhub_name}": [0]}}', # to read from specific partitions use option: "assign": '{"topic_name": [0, 1]}'
    # "startingOffsets": f'{{"{eventhub_name}": {{"0": 212350}}}}', # "latest", "earliest", '{"topic_name: {"0": 62821}}' -1: latest, -2: earliest

    "enable.auto.commit": "true ",
    "groupIdPrefix": "Stream_Analytics_",
    "auto.commit.interval.ms": "5000"
}

# now passenger‐requests consumer config, pointing at the other hub
kafkaConfDrivers = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism":     "PLAIN",
    "kafka.security.protocol":  "SASL_SSL",
    # this password must include EntityPath=group1_passenger_requests
    "kafka.sasl.jaas.config":   f'org.apache.kafka.common.security.plain.PlainLoginModule required \
username="$ConnectionString" \
password="{consumer_eventhub_connection_str_requests}";',
    "subscribe":                "drivers-g8-v2",
    "startingOffsets":          "latest",
    "enable.auto.commit":       "true",
    "groupIdPrefix":            "Stream_Analytics_",
    "auto.commit.interval.ms":  "5000"
}

print(kafkaConfRides_Req)
print(kafkaConfDrivers)

{'kafka.bootstrap.servers': 'iesstsabbadbaa-grp-06-10.servicebus.windows.net:9093', 'kafka.sasl.mechanism': 'PLAIN', 'kafka.security.protocol': 'SASL_SSL', 'kafka.sasl.jaas.config': 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=ridesconsumer;SharedAccessKey=f82trXEoV+VyuCL2Xah3fSOB9JWoaG36L+AEhJ9CqaE=;EntityPath=ride-requests-g8-v2";', 'subscribe': 'ride-requests-g8-v2', 'startingOffsets': 'latest', 'enable.auto.commit': 'true ', 'groupIdPrefix': 'Stream_Analytics_', 'auto.commit.interval.ms': '5000'}
{'kafka.bootstrap.servers': 'iesstsabbadbaa-grp-06-10.servicebus.windows.net:9093', 'kafka.sasl.mechanism': 'PLAIN', 'kafka.security.protocol': 'SASL_SSL', 'kafka.sasl.jaas.config': 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;S

In [ ]:
# Read from Event Hub using Kafka
df_ride_request = spark \
    .readStream \
    .format("kafka") \
    .options(**kafkaConfRides_Req)

# Read from Event Hub using Kafka
df_drivers = spark \
    .readStream \
    .format("kafka") \
    .options(**kafkaConfDrivers)

In [ ]:
df_ride_request= df_ride_request.load()  # Start reading data from the specified Kafka topic
df_ride_request.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
df_drivers= df_drivers.load()  # Start reading data from the specified Kafka topic
df_drivers.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
from pyspark.sql.avro.functions import from_avro
# Deserialize the AVRO messages from the value column
df_ride_request = df_ride_request.select(from_avro(df_ride_request.value, ride_request_schema).alias("ride-requests-g8-v2"))

# Print the schema of the DataFrame
df_ride_request.printSchema()

root
 |-- ride-requests-g8-v2: struct (nullable = true)
 |    |-- request_id: string (nullable = false)
 |    |-- passenger_id: string (nullable = false)
 |    |-- passenger_name: string (nullable = false)
 |    |-- driver_name: string (nullable = true)
 |    |-- pickup_latitude: double (nullable = false)
 |    |-- pickup_longitude: double (nullable = false)
 |    |-- destination_latitude: double (nullable = false)
 |    |-- destination_longitude: double (nullable = false)
 |    |-- timestamp: string (nullable = false)
 |    |-- vehicle_type: string (nullable = false)
 |    |-- surge_multiplier: float (nullable = false)
 |    |-- traffic_delay_multiplier: float (nullable = false)
 |    |-- pickup_distance_km: float (nullable = false)
 |    |-- pickup_eta_min: float (nullable = false)
 |    |-- distance_km: float (nullable = false)
 |    |-- estimated_ride_time_min: float (nullable = false)
 |    |-- fare_euros: float (nullable = false)
 |    |-- payment_method: string (nullable = false

In [ ]:
df_drivers = df_drivers.select(from_avro(df_drivers.value, driver_profile_schema).alias("drivers-g8-v2"))

# Print the schema of the DataFrame
df_drivers.printSchema()

root
 |-- drivers-g8-v2: struct (nullable = true)
 |    |-- driver_id: string (nullable = false)
 |    |-- driver_name: string (nullable = false)
 |    |-- vehicle_type: string (nullable = false)
 |    |-- latitude: double (nullable = false)
 |    |-- longitude: double (nullable = false)
 |    |-- status: string (nullable = false)
 |    |-- shift_start: integer (nullable = false)
 |    |-- shift_duration: integer (nullable = false)



In [ ]:
from pyspark.sql.functions import col

df_ride_request_flat = df_ride_request.select(
    col("ride-requests-g8-v2.request_id"),
    col("ride-requests-g8-v2.passenger_id"),
    col("ride-requests-g8-v2.passenger_name"),
    col("ride-requests-g8-v2.driver_name"),
    col("ride-requests-g8-v2.pickup_latitude"),
    col("ride-requests-g8-v2.pickup_longitude"),
    col("ride-requests-g8-v2.destination_latitude"),
    col("ride-requests-g8-v2.destination_longitude"),
    col("ride-requests-g8-v2.timestamp"),
    col("ride-requests-g8-v2.vehicle_type"),
    col("ride-requests-g8-v2.surge_multiplier"),
    col("ride-requests-g8-v2.traffic_delay_multiplier"),
    col("ride-requests-g8-v2.pickup_distance_km"),
    col("ride-requests-g8-v2.pickup_eta_min"),
    col("ride-requests-g8-v2.distance_km"),
    col("ride-requests-g8-v2.estimated_ride_time_min"),
    col("ride-requests-g8-v2.fare_euros"),
    col("ride-requests-g8-v2.payment_method"),
    col("ride-requests-g8-v2.driver_rating_given")
)

df_ride_request_flat.printSchema()


root
 |-- request_id: string (nullable = true)
 |-- passenger_id: string (nullable = true)
 |-- passenger_name: string (nullable = true)
 |-- driver_name: string (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- destination_latitude: double (nullable = true)
 |-- destination_longitude: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- surge_multiplier: float (nullable = true)
 |-- traffic_delay_multiplier: float (nullable = true)
 |-- pickup_distance_km: float (nullable = true)
 |-- pickup_eta_min: float (nullable = true)
 |-- distance_km: float (nullable = true)
 |-- estimated_ride_time_min: float (nullable = true)
 |-- fare_euros: float (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- driver_rating_given: float (nullable = true)



In [ ]:
from pyspark.sql.functions import col

df_drivers_flat = df_drivers.select(
    col("drivers-g8-v2.driver_id"),
    col("drivers-g8-v2.driver_name"),
    col("drivers-g8-v2.vehicle_type"),
    col("drivers-g8-v2.latitude"),
    col("drivers-g8-v2.longitude"),
    col("drivers-g8-v2.status"),
    col("drivers-g8-v2.shift_start"),
    col("drivers-g8-v2.shift_duration")
)

df_drivers_flat.printSchema()


root
 |-- driver_id: string (nullable = true)
 |-- driver_name: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- status: string (nullable = true)
 |-- shift_start: integer (nullable = true)
 |-- shift_duration: integer (nullable = true)



In [ ]:
# Ride‑requests sink paths
#ride_requests_output_path      = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/ride-requests-output/"
#ride_requests_checkpoint_path  = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/ride-requests-checkpoint/"

# Driver‑profiles sink paths
#drivers_output_path            = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/drivers-output/"
#drivers_checkpoint_path        = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/drivers-checkpoint/"

#ride_requests_output_path, ride_requests_checkpoint_path, drivers_output_path, drivers_checkpoint_path

In [ ]:
ride_requests_output_path = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/ride-requests-output/"
ride_requests_checkpoint_path = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/ride-requests-checkpoint/"

drivers_output_path   = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/drivers-output/"
drivers_checkpoint_path = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/drivers-checkpoint/"
ride_requests_output_path, ride_requests_checkpoint_path, drivers_output_path, drivers_checkpoint_path

('abfss://streamed-data-group8@iesstsabbadbaa.dfs.core.windows.net/ride-requests-output/',
 'abfss://streamed-data-group8@iesstsabbadbaa.dfs.core.windows.net/ride-requests-checkpoint/',
 'abfss://streamed-data-group8@iesstsabbadbaa.dfs.core.windows.net/drivers-output/',
 'abfss://streamed-data-group8@iesstsabbadbaa.dfs.core.windows.net/drivers-checkpoint/')

In [ ]:
# Ride‑requests streaming write (con nombre de query)
#ride_requests_query = df_ride_request_flat.writeStream \
#    .format("parquet") \
#    .option("checkpointLocation", ride_requests_checkpoint_path) \
#    .option("path", ride_requests_output_path) \
#    .queryName("ride_requests_to_blob") \
#    .trigger(processingTime="5 seconds") \
#    .start()

# Driver‑profiles streaming write (con nombre de query)
#drivers_query = df_drivers_flat.writeStream \
#    .format("parquet") \
#    .option("checkpointLocation", drivers_checkpoint_path) \
#    .option("path", drivers_output_path) \
#    .queryName("drivers_to_blob") \
#    .trigger(processingTime="5 seconds") \
#    .start()


In [ ]:
# Get the list of active streaming queries
#active_queries = spark.streams.active

# Print details about each active query
#for query in active_queries:
#    print(f"Query Name: {query.name}")
#    print(f"Query ID: {query.id}")
#    print(f"Query Status: {query.status}")
#    print(f"Is Query Active: {query.isActive}")
#    print("-" * 50)


In [ ]:
ride_requests_query = df_ride_request_flat.writeStream \
    .format("parquet") \
    .option("checkpointLocation", ride_requests_checkpoint_path) \
    .option("path", ride_requests_output_path) \
    .queryName("ride_requests_to_blob") \
    .trigger(processingTime="5 seconds") \
    .start()


drivers_query = df_drivers_flat.writeStream \
    .format("parquet") \
    .option("checkpointLocation", drivers_checkpoint_path) \
    .option("path", drivers_output_path) \
    .queryName("drivers_to_blob") \
    .trigger(processingTime="5 seconds") \
    .start()

import time
for i in range(24):
    time.sleep(5)
    print(f"  ⏳ {5*(i+1)}s transcurridos, queries activas:",
          [q.name for q in spark.streams.active])


  ⏳ 5s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 10s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 15s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 20s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 25s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 30s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 35s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 40s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 45s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 50s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 55s transcurridos, queries activas: ['ride_requests_to_blob', 'drivers_to_blob']
  ⏳ 60s transcurridos, queries activas: ['ride_requests_to_blob', 

In [ ]:
# Lee todo el directorio Parquet
df = spark.read.parquet(ride_requests_output_path)

# Muestra las primeras filas de forma legible
df.show(20, truncate=False)

# O, si quieres extraer a Pandas:
#pdf = df.limit(100).toPandas()


+------------------------------------+------------------------------------+---------------------------------+------------------------+---------------+----------------+--------------------+---------------------+-------------------+------------+----------------+------------------------+------------------+--------------+-----------+-----------------------+----------+--------------+-------------------+
|request_id                          |passenger_id                        |passenger_name                   |driver_name             |pickup_latitude|pickup_longitude|destination_latitude|destination_longitude|timestamp          |vehicle_type|surge_multiplier|traffic_delay_multiplier|pickup_distance_km|pickup_eta_min|distance_km|estimated_ride_time_min|fare_euros|payment_method|driver_rating_given|
+------------------------------------+------------------------------------+---------------------------------+------------------------+---------------+----------------+--------------------+--------

# ANALYSIS

In [ ]:
import os
import subprocess

# Fetch the latest Spark 3.x.x version
# curl -s https://downloads.apache.org/spark/ → Fetches the Spark download page.
# grep -o 'spark-3\.[0-9]\+\.[0-9]\+' → Extracts only versions that start with spark-3. (ignoring Spark 4.x if it exists in the future).
# sort -V → Sorts the versions numerically.
# tail -1 → Selects the latest version.
spark_version = subprocess.run(
    "curl -s https://downloads.apache.org/spark/ | grep -o 'spark-3\\.[0-9]\\+\\.[0-9]\\+' | sort -V | tail -1",
    shell=True, capture_output=True, text=True
).stdout.strip()

spark_version

'spark-3.5.5'

In [ ]:
spark_release=spark_version
hadoop_version='hadoop3'

import os, time
start=time.time()
os.environ['SPARK_RELEASE']=spark_release
os.environ['HADOOP_VERSION']=hadoop_version
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_release}-bin-{hadoop_version}"

In [ ]:
if True:
  # Run below commands in google colab
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null # install Java8
  !wget -q http://apache.osuosl.org/spark/${SPARK_RELEASE}/${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # download spark-3.3.X
  !tar xf ${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # unzip it

  !pip install -q findspark # install findspark
  # findspark find your Spark Distribution and sets necessary environment variables

In [ ]:
import findspark
findspark.init()

# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.5.5


In [ ]:
# 1) SparkSession + Azure Blob config
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, count, avg

account_name = "iesstsabbadbaa"
account_key  = "YiUournbsPdwTGVanY1bNSsUcy1Z7NUNoty1fo0SDxcHoe6gzTxeq7BlIG4c3owhT1IIhim2IsHF+AStMie8jw=="
container_name = "streamed-data-group8"

spark = SparkSession \
    .builder \
    .appName("RideHailingAnalytics") \
    .config("spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,"
        "org.apache.spark:spark-avro_2.12:3.5.0,"
        "org.apache.hadoop:hadoop-azure:3.3.1,"
        "com.microsoft.azure:azure-storage:8.6.6") \
    .config(f"fs.azure.account.key.{account_name}.dfs.core.windows.net", account_key) \
    .getOrCreate()


In [ ]:

# 2) Paths to your Parquet output on ADLS Gen2
ride_requests_output_path = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/ride-requests-output/"

# 3) Quick batch‐mode sanity check
rides_df = spark.read.parquet(ride_requests_output_path)
rides_df.printSchema()
rides_df.show(5, truncate=False)

root
 |-- request_id: string (nullable = true)
 |-- passenger_id: string (nullable = true)
 |-- passenger_name: string (nullable = true)
 |-- driver_name: string (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- destination_latitude: double (nullable = true)
 |-- destination_longitude: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- surge_multiplier: float (nullable = true)
 |-- traffic_delay_multiplier: float (nullable = true)
 |-- pickup_distance_km: float (nullable = true)
 |-- pickup_eta_min: float (nullable = true)
 |-- distance_km: float (nullable = true)
 |-- estimated_ride_time_min: float (nullable = true)
 |-- fare_euros: float (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- driver_rating_given: float (nullable = true)

+------------------------------------+------------------------------------+-----------------------+------

In [ ]:
# 2b) Path to your Driver‑profiles Parquet output on ADLS Gen2
drivers_output_path = f"abfss://{container_name}@{account_name}.dfs.core.windows.net/drivers-output/"

# 3b) Quick batch‑mode sanity check for drivers
drivers_df = spark.read.parquet(drivers_output_path)
drivers_df.printSchema()
drivers_df.show(5, truncate=False)


root
 |-- driver_id: string (nullable = true)
 |-- driver_name: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- status: string (nullable = true)
 |-- shift_start: integer (nullable = true)
 |-- shift_duration: integer (nullable = true)

+------------------------------------+----------------------+------------+---------+---------+---------+-----------+--------------+
|driver_id                           |driver_name           |vehicle_type|latitude |longitude|status   |shift_start|shift_duration|
+------------------------------------+----------------------+------------+---------+---------+---------+-----------+--------------+
|9adfb4ec-f1d0-49f4-985f-55c3995ae05b|Eusebio Bernal Vara   |Standard    |40.428378|-3.69141 |available|12         |8             |
|5b172182-b625-4918-ad14-56a0d178ecf4|Dominga Plaza Ávila   |Luxury      |40.437566|-3.678017|available|6          |8           

In [ ]:
!pip install azure-storage-blob

In [ ]:
from azure.storage.blob import BlobServiceClient

AZURE_STORAGE_CONNECTION_STRING = (
    "DefaultEndpointsProtocol=https;"
    "AccountName=iesstsabbadbaa;"
    "AccountKey=YiUournbsPdwTGVanY1bNSsUcy1Z7NUNoty1fo0SDxcHoe6gzTxeq7BlIG4c3owhT1IIhim2IsHF+AStMie8jw==;"
    "EndpointSuffix=core.windows.net"
)
CONTAINER = "streamed-data-group8"
svc = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
ctr = svc.get_container_client(CONTAINER)

print("=== ride-requests-output ===")
for b in ctr.list_blobs(name_starts_with="ride-requests-output/"):
    print(" ", b.name)

print("\n=== drivers-output ===")
for b in ctr.list_blobs(name_starts_with="drivers-output/"):
    print(" ", b.name)


=== ride-requests-output ===
  ride-requests-output/_spark_metadata
  ride-requests-output/_spark_metadata/0
  ride-requests-output/_spark_metadata/1
  ride-requests-output/_spark_metadata/10
  ride-requests-output/_spark_metadata/11
  ride-requests-output/_spark_metadata/12
  ride-requests-output/_spark_metadata/13
  ride-requests-output/_spark_metadata/14
  ride-requests-output/_spark_metadata/15
  ride-requests-output/_spark_metadata/16
  ride-requests-output/_spark_metadata/17
  ride-requests-output/_spark_metadata/18
  ride-requests-output/_spark_metadata/19.compact
  ride-requests-output/_spark_metadata/2
  ride-requests-output/_spark_metadata/20
  ride-requests-output/_spark_metadata/21
  ride-requests-output/_spark_metadata/22
  ride-requests-output/_spark_metadata/3
  ride-requests-output/_spark_metadata/4
  ride-requests-output/_spark_metadata/5
  ride-requests-output/_spark_metadata/6
  ride-requests-output/_spark_metadata/7
  ride-requests-output/_spark_metadata/8
  ride-re

## RUN TO STOP SPARK SESSION

In [ ]:
# Stop any active streaming queries
for q in spark.streams.active:
    print(f"Stopping query {q.name}…")
    q.stop()

# Finally, stop the SparkSession
spark.stop()
print("✅ Spark session stopped.")
